#  T-tests and P-values

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_rel`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](http://b.link/test50) and [this Quora](http://b.link/unpaired97). Make sure you understand when to use which type of t-test. 

In [35]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from scipy.stats import ttest_1samp

### One tailed t-test 
In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/ttest_machine.xlsx. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

#### Import dataset

In [36]:
machine = pd.read_csv("ttest_machine.txt", delimiter=' ')

In [37]:
machine

,New_machine,Old_machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [38]:
# H0 = mean(new_machine) <= mean(old_machine)
# H1 = mean(new_machine) > mean(old_machine)

In [39]:
st.ttest_rel(machine['New_machine'], machine['Old_machine'])

TtestResult(statistic=-3.0614273841115844, pvalue=0.013540335651632521, df=9)

In [40]:
# the pvalue is close to 0 and <0,05
# but the stat is negatif (sign of the H0 goes in the same direction) which means the mean_new is < mean_old
# we can accept the H0

In [41]:
print('mean_new ', machine.New_machine.mean())
print('mean_old ', machine.Old_machine.mean())

mean_new  42.14
mean_old  43.230000000000004


#### Import dataset

In this challenge we will work on the Pokemon dataset you have already used. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.). Use pokemon.csv

In [42]:
pokemon = pd.read_csv("pokemon.txt", delimiter=',')
print(pokemon.shape)
pokemon.head(5)

(800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [43]:
pokemon_list = list(pokemon['Type 1'].unique())
pokemon_list

['Grass',
 'Fire',
 'Water',
 'Bug',
 'Normal',
 'Poison',
 'Electric',
 'Ground',
 'Fairy',
 'Fighting',
 'Psychic',
 'Rock',
 'Ghost',
 'Ice',
 'Dragon',
 'Dark',
 'Steel',
 'Flying']

In [44]:
sample_1 = pokemon[pokemon.Legendary==True].sample(30)
display(sample_1)
sample_2 = pokemon[pokemon.Legendary==False].sample(30)
sample_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
706,643,Reshiram,Dragon,Fire,680,100,120,100,150,120,90,5,True
418,380,LatiasMega Latias,Dragon,Psychic,700,80,100,120,140,150,110,3,True
417,380,Latias,Dragon,Psychic,600,80,80,90,110,130,110,3,True
425,384,Rayquaza,Dragon,Flying,680,105,150,90,150,90,95,3,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
552,493,Arceus,Normal,NaN,720,120,120,120,120,120,120,4,True
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True
705,642,ThundurusTherian Forme,Electric,Flying,580,79,105,70,145,80,101,5,True
422,382,KyogrePrimal Kyogre,Water,NaN,770,100,150,90,180,160,90,3,True


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
358,327,Spinda,Normal,NaN,360,60,60,60,60,60,60,3,False
673,612,Haxorus,Dragon,NaN,540,76,147,90,60,70,97,5,False
581,522,Blitzle,Electric,NaN,295,45,60,32,50,32,76,5,False
436,391,Monferno,Fire,Fighting,405,64,78,52,78,52,81,4,False
444,399,Bidoof,Normal,NaN,250,59,45,40,35,40,31,4,False
769,699,Aurorus,Rock,Ice,521,123,77,72,99,92,58,6,False
567,508,Stoutland,Normal,NaN,500,85,110,90,45,90,80,5,False
630,569,Garbodor,Poison,NaN,474,80,95,82,60,82,75,5,False
46,41,Zubat,Poison,Flying,245,40,45,35,30,40,55,1,False
387,354,BanetteMega Banette,Ghost,NaN,555,64,165,75,93,83,75,3,False


In [45]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    for i in features:
        results[i]=st.ttest_ind(s1[i], s2[i], equal_var=False)[1]
    
    return results

In [46]:
t_test_features(sample_1, sample_2)

{'HP': 5.778275220225359e-07,
 'Attack': 2.8456069425467076e-06,
 'Defense': 2.2731266571978813e-07,
 'Sp. Atk': 1.9374494405867824e-15,
 'Sp. Def': 7.683405295552121e-08,
 'Speed': 4.475414242869621e-09,
 'Total': 3.3773083278701866e-17}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [47]:
# Your comment here

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [48]:
generation_1 = pokemon[pokemon.Generation==1].sample(30)
generation_2 = pokemon[pokemon.Generation==2].sample(30)

In [49]:
t_test_features(generation_1, generation_2)

{'HP': 0.1597399405764335,
 'Attack': 0.4969447487174913,
 'Defense': 0.6609978079765477,
 'Sp. Atk': 0.1350557621710817,
 'Sp. Def': 0.8329856691674644,
 'Speed': 0.06935844109246704,
 'Total': 0.609196929199088}

#### What conclusions can you make?

In [50]:
# Your comment here

#### Compare pokemons who have single type vs those having two types.

In [51]:
type_1 = pokemon[pokemon['Type 2'].isnull()].sample(30)
type_2 = pokemon[~pokemon['Type 2'].isnull()].sample(30)

#### What conclusions can you make?

In [52]:
t_test_features(type_1, type_2)

{'HP': 0.5014445638428389,
 'Attack': 0.07435278839347739,
 'Defense': 0.016947428486625348,
 'Sp. Atk': 0.2023676369872362,
 'Sp. Def': 0.0010945326029847622,
 'Speed': 0.27971409408186,
 'Total': 0.008811100776035795}

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [54]:
st.ttest_rel(pokemon['Attack'], pokemon['Defense'])


TtestResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

#### What conclusions can you make?

In [55]:
st.ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

TtestResult(statistic=0.853986188453353, pvalue=0.3933685997548122, df=799)